# Paper Numerical Results

Step 1: Install DASK etc (follow instructions in Dask.ipynb)

Step 2: Launch DASK server:
 
    dask-ec2 up --keyname research --keypair .ssh/research.pem --region-name eu-west-1 --ami ami-d37961b5 --tags research:dp --count 4 --volume-size 30 --type c4.8xlarge

Step 3: On the scheduler, run:

    pip install git+https://github.com/lionfish0/dask_dp4gp.git
    
Step 4: On the scheduler, run this notebook. In particular run this cell once: 

In [1]:
import dask_dp4gp

dask_dp4gp.install_libraries_on_workers()

Next run the following analysis:

In [ ]:
from dp4gp.utils import dp_normalise, dp_unnormalise
from dp4gp import datasets
from dask.distributed import Client
from sklearn.model_selection import KFold
import dask_dp4gp

client = Client('127.0.0.1:8786')

####Set up data and parameter search grid
kung = datasets.load_kung()
sensitivity = 100.0
y,ac_sens,norm_params = dp_normalise(kung[kung[:,3]==0,0:1],sensitivity)
X = kung[kung[:,3]==0,1:3]

#todo these don't do anything - shift to get used a p_grid items
epsilon = 1.0
delta = 0.01
cv = 3
p_grid = {"lengthscale":[], 'variance':[]}#, 'noisevariance':[]}
for ls in 5.0**np.arange(0,2):
    p_grid["lengthscale"].append(ls)
for v in 5.0**np.arange(-1,1):
    p_grid["variance"].append(v)
    
####Get the -RMSE for each fold/param-combo
scores = dask_dp4gp.getscores(X,y,p_grid,cv)

kf = KFold(n_splits=cv)
probabilities = []
for train_index, test_index in kf.split(X):
    X_train = X[train_index]
    y_train = y[train_index]
    probabilities.append(dask_dp4gp.getprobabilities(X_train,y_train,p_grid,5))
    
print(np.array(probabilities))
print(scores)
print(np.sum(probabilities*-scores,1))
print(np.mean(-scores,1))